# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [3]:
from unityagents import UnityEnvironment
import numpy as np
import pandas as pd
import random
import importlib
import time
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import ddpg_agent
from helper import *
RANDOM_SEED = 42

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64', seed=RANDOM_SEED)

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64', seed=RANDOM_SEED)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -7.79496002e+00  -1.00000000e+00
   1.79961014e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   1.64491057e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [6]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.23999999463558197


This cell is the same as above, but using a ddpg agent.

In [14]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#Create agents
agents = ddpg_agent.Agent(state_size, action_size, num_agents, RANDOM_SEED)
agents.reset()                                         # reset the agent noise function to mu
                                 
t = 0                                                  # initialize number of timesteps
while True:
    # select an action (for each agent)
    actions = agents.act(states)               
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to the environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    # Save the experience and learn
    agents.step(states, actions, rewards, next_states, dones)    
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    t+=1                                               # update number of time steps
    print('\rTime Steps: {}'.format(t), end="")
    if np.any(dones):                                  # exit loop if episode finished
        break
print('\nTotal score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Time Steps: 1001
Total score (averaged over agents) this episode: 0.2474999944679439


When finished, you can close the environment.

In [15]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

### 5. Train the Agent with DDPG  

In [16]:
agents = ddpg_agent.Agent(state_size, action_size, num_agents, RANDOM_SEED)
len(agents.memory)

0

In [17]:
# Modified from Navigation

def ddpg(n_episodes=175, max_t=1100, scores_window_len=100, goal_score = 30, 
         print_every = 10, save_every=10, resume_training=False):
    """Deep Q-Learning.
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        scores_window_len: a rolling window size for averaging the scores
        goal_score: the minimum average score required for a satisfactory model
        save_every: how often the models are saved
        resume_training: whether or not to load a previous model from a checkpoint
    """
    agents = ddpg_agent.Agent(state_size, action_size, num_agents, RANDOM_SEED)  # create agents
    
    scores_per_episode = []                                  # List containing agent scores from each episode
    scores_window = deque(maxlen=scores_window_len)          # last however many scores
    
    # Load a checkpoint
    if resume_training:
        agents, scores_per_episode, scores_window, = load_checkpoint(agents, scores_per_episode,
                                                                            scores_window)  
    
    for i_episode in range(len(scores_per_episode)+1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]    # reset the environment
        states = env_info.vector_observations                # get the current states
        scores = np.zeros(num_agents)                        # initialize the score (for each agent)
        agents.reset()                                       # reset internal noise state to mu
        for t in range(1, max_t+1):          
            actions = agents.act(states)                     # agents act
            env_info = env.step(actions)[brain_name]         # step the environment
            next_states = env_info.vector_observations       # observe next state
            rewards = env_info.rewards                       # observe rewards
            dones = env_info.local_done                      # observe dones             
                                                             # save the experience and learn
            agents.step(states, actions, rewards, next_states, dones)
            states = next_states                             # roll over states to next time step
            scores += rewards                                # update the score (for each agent)
                                                             # display time steps and mean of all agent scores
            print('\rEpisode {}\tScore: {:.2f}\tTime Step {}\tSize of memory: {}'
                  .format(i_episode, np.mean(scores), t, len(agents.memory)), end=" "*5)
            if np.any(dones):                                # exit loop if episode finished
                break    

        scores_window.append(scores)                         # save most recent scores
        scores_per_episode.append(scores)                    # save most recent scores        
        if i_episode % print_every == 0:                     # output average of scores_windows
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_per_episode[-print_every:])), 
                  end=" "*35+"\n")    
        if np.mean(scores_window)>=goal_score:               # proclaim the evironment solved, save checkpoints 
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'
                  .format(i_episode-scores_window_len, np.mean(scores_window)))
            save_checkpoint(agents, scores_per_episode, ddpg_agent, 'checkpoint.tar')
            break
        if i_episode % save_every ==0:                        # save the models
            save_checkpoint(agents, scores_per_episode, ddpg_agent, 'checkpoint.tar')
    return scores_per_episode

In [18]:
def run_agent(n_episodes=175, max_t=1001, scores_window_len=100, goal_score = 30, 
              print_every=10, save_every=10,
              resume_training=False, plot=False, title='Scores'):
    """ Runs an agent over episodes and returns the scores from each episode  
        Attempts to make hyperparameter tuning easier
        Updates for the changes in the dqn_agent.py file 
        scores_window_len: a rolling window size for averaging the scores
        goal_score: the minimum average score required for a satisfactory model
        verbose: 0-No output, 1-Score and Time Elapsed, 2-Score, Time Elapsed and Hyperparameter
        plot: whether or not to plot the scores
        title: plot title
    """
    
    # reload library after modifying Hyperparameters
    importlib.reload(ddpg_agent)

    # Generate Scores
    tic = time.time()
    print_hyperparameters(ddpg_agent)       
    scores = ddpg(n_episodes, max_t, scores_window_len, goal_score, print_every, save_every, resume_training)
    toc = time.time()
    print("\nTime elapsed: {:.0f} seconds".format(toc-tic))

    # Plot Scores
    if plot == True:
        plot_scores(scores, rolling_window=scores_window_len, title=title)

    return scores

In [19]:
# import sys, importlib
# importlib.reload(sys.modules['helper'])
# from helper import load_checkpoint

In [ ]:
scores = run_agent(n_episodes=10, max_t=1500, scores_window_len=100, goal_score = 30, 
                   print_every=10, save_every=5,
                   resume_training=True, plot=True, title='learn every step')

BUFFER_SIZE = 1000000
BATCH_SIZE = 128
GAMMA = 0.99
TAU = 0.001
LR_ACTOR = 0.0003
LR_CRITIC = 0.001
WEIGHT_DECAY = 0.0
UPDATE_EVERY = 20
N_UPDATES = 10
FC1_UNITS_ACTOR = 256
FC2_UNITS_ACTOR = 128
FCS1_UNITS_CRITIC = 256
FC2_UNITS_CRITIC = 128
Episode 9	Score: 0.29	Time Step 356	Size of memory: 167280      

When finished, you may close the environment

In [21]:
# env.close()
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64', seed=RANDOM_SEED)
